## Como formatar as respostas do GPT utilizando a biblioteca LangChain

> Nota: também disponível no [Medium.com](https://medium.com/@erickrribeiro/uso-de-resultados-estruturados-e-formatados-com-gpt-e-langchain-f7d57d1f28c9)


## Pré-requisitos

Para executar os códigos-fonte apresentados nesse notebook é necessário instalar as seguintes bibliotecas:

In [ ]:
!pip install langchain==0.0.262

In [ ]:
!pip install openai==0.27.8

In [20]:
import openai
import langchain

print("OpenAI:", openai.__version__)
print("LangChain:", langchain.__version__)

OpenAI: 0.27.8
LangChain: 0.0.304


## Chave de API da OpenAI

O LLM da OpenAI é um serviço pago. Então, para utilizá-lo é necessário criar uma conta na OpenAI e gerar uma chave de API. Para mais detalhes, consulte o [link](https://openai.com/blog/openai-api). Após obter a chave, crie uma variável de ambiente da seguinte forma:

```bash
export OPENAI_API_KEY=<SUBSTITUA_PELA_CHAVE>
```

In [19]:
import os

from dotenv import find_dotenv, load_dotenv

# Também é possível carregar um arquivo .ev
_ = load_dotenv(find_dotenv())


## Prompt Template

O prompt é o texto de entrada que deve ser enviado para o LLM, e serve para guiar o LLM na execução de uma tarefa. Durante a construção de um prompt é possível definir uma persona, incluir exemplos, instruções e regras. O modelo usa as informações contidas no prompt para compreender o contexto e gerar respostas coerentes. O LangChain fornece várias classes e funções para facilitar a construção de prompts.


In [22]:
from langchain.prompts import PromptTemplate


template = "Sua tarefa é corrigir erros de ortografia.\nCorrija o texto: '{text}'"
prompt_template = PromptTemplate(
  input_variables=["text"],
  template=template)
prompt = prompt_template.format(
  text="Estou certo   de que esse artigo vai me ajuudar no trnaalho")

print(prompt)

Sua tarefa é corrigir erros de ortografia.
Corrija o texto: 'Estou certo   de que esse artigo vai me ajuudar no trnaalho'


## Prompt Template + LLM

In [23]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = "Sua tarefa é corrigir erros de ortografia.\nCorrija o texto: '{text}'"
prompt_template = PromptTemplate(
  input_variables=["text"],
  template=template)

llm = OpenAI()
chain = LLMChain(llm=llm, prompt=prompt_template)
response = chain.run({"text": "Estou certo   de que esse artigo vai me ajuudar no trnaalho"})

print("Tipo da Resposta: ", type(response))
print("Texto Corrigido:", response)

Tipo da Resposta:  <class 'str'>
Texto Corrigido: 

Estou certo de que este artigo vai me ajudar no trabalho.


## Explicações

Note que a resposta do LLM é um texto sem erros de digitação. Agora, vamos analisar como é feito o encadeamento do Prompt com o LMM da OpenAI:
- Primeiro, é criada uma instância do modelo de linguagem da OpenAI, com o trecho llm = OpenAI()
- Depois, é criada uma instância da classe LLMChain. Como já foi dito, um Chain liga dois ou mais componentes. Na ocasião, a Chain é criada com o objetivo de conectar o PromptTemplate ao LLM da OpenAI.
- Finalizando, é chamado o método run(), no trecho chain.run. Note, que o texto a ser corrigido é passado como um dicionário na chamada do método. Além disso, é utilizadas a parâmetro text, definido na classe PromptTemplate, no trecho input_variables=["text"].

## Prompt Template + LLM + OutputParser

In [25]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.output_parsers import (
    ResponseSchema,
    StructuredOutputParser,
)
from langchain.prompts import PromptTemplate


# inicia modelo
llm = OpenAI()

# definir o formato da resposta
response_schemas = [
    ResponseSchema(name="original_text", description="texto de entrada no formato bruto."),
    ResponseSchema(name="corrected_text", description="texto após correção ortográfica.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


# Definicao do Prompt
template = "Sua tarefa é corrigir erros de ortografia.{format_instructions}\nCorrija o texto: '{text}'"
prompt_template = PromptTemplate(
  input_variables=["text"],
  template=template,
  partial_variables={"format_instructions": format_instructions})


chain = LLMChain(llm=llm, prompt=prompt_template)
response = chain.run({"text": "Estou certo   de que esse artigo vai me ajuudar no trnaalho"})
response = output_parser.parse(response)

response

{'original_text': 'Estou certo de que esse artigo vai me ajuudar no trnaalho',
 'corrected_text': 'Estou certo de que esse artigo vai me ajudar no trabalho'}

In [26]:
response["corrected_text"]

'Estou certo de que esse artigo vai me ajudar no trabalho'

In [ ]:
response["corrected_text"]

## Conclusão

Nesse artigo, foi mostrado como formatar as respostas geradas pelo modelo GPT, com uso da biblioteca LangChain. O ato de formatar as respostas pode ajudar desenvolvedores que trabalham com LLMs e buscam não apenas a saída no formato de texto livre, mas também respostas formatadas como classes, dicionários ou listas. Continue experimentando, construindo e criando, pois a jornada para uma integração mais profunda e eficaz com modelos de linguagem está apenas começando. A "LangChain" e tecnologias semelhantes estão aqui para capacitar sua imaginação e permitir que você transforme ideias em realidade de maneiras que nunca antes foram possíveis.